In [2]:
import json
import pandas as pd
import dill as pickle

class DataUtil2:
    """"""
    @classmethod
    def load(cls, file, format, date_cols= None):
        """"""
        data = ''
        if(format== 'csv'):
            data = pd.read_csv(file, parse_dates= date_cols)
        elif(format== 'json'):
            with open(file, 'r') as i_file:
                data = json.load(file)
            i_file.close()
        elif(format== 'pkl'):
            with open(file, 'rb') as i_file:
                data = pickle.load(i_file)
            i_file.close()
        elif(format == 'hdf'):
            data = pd.read_hdf(path_or_buf= file, key='undefined')

        return  data

    @classmethod
    def save(cls, data, file, format, precision= 8):
        """"""
        if(format == 'csv'):
            data.to_csv(file, float_format= '%%.%df' % precision, index= False)
        elif(format == 'json'):
            with open(file, 'w') as o_file:
                json.dump(data, o_file, ensure_ascii= True, indent= 4)
            o_file.close()
        elif(format == 'pkl'):
            with open(file, 'wb') as o_file:
                pickle.dump(data, o_file, -1)
            o_file.close()
        elif(format== 'hdf'):
            data.to_hdf(path_or_buf= file, key='undefined', mode='w', complib='blosc')

        return




In [3]:
#######################
# LigthGBM Regression #
#######################
import numpy as np
import pandas as pd
import datetime
import time
import os,sys
import gc
from sklearn import *
import lightgbm

drop_cols = ['id', 'visit_date', 'visitors', 'hpg_store_id', 'fold', 'air_store_id']
 
strategy = 'lgb_l2'
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5

DataBaseDir = '../../../data/LOO'
HoldoutInputDir = '%s/l0/holdout' % DataBaseDir
OutputDir = '%s/l1/kfold' % DataBaseDir

TrainData = pd.read_csv('%s/train.csv' % HoldoutInputDir)
HoldoutData = pd.read_csv('%s/holdout.csv' % HoldoutInputDir)
##### model selection with CV
# score
cv_score = .0
holdout_score = .0
# predict
y_test_pred = 0
# parameters
params = {
    "boosting": "gbdt",
    "objective": "regression_l2",
    "lambda_l2": 200,
    
    "num_iterations": 150,
    "learning_rate": 0.20,
    "min_data_in_leaf": 20,
    'num_leaves': 255,
    "max_depth": 25,

    "feature_fraction": 0.90,
    "bagging_fraction": 0.85,
    "bagging_freq": 20,
    "min_hessian": 0.001,

    "max_bin": 63,
}

col = [c for c in TrainData.columns if c not in drop_cols]
# train
d_cv = lightgbm.Dataset(TrainData[col], label= TrainData['visitors'].values, max_bin= params['max_bin'], silent= True, free_raw_data= True)
model = lightgbm.train(params, d_cv)
# for holdout
HoldoutData[strategy] = model.predict(HoldoutData[col])
rmsle_holdout = RMSLE(HoldoutData['visitors'].values, HoldoutData[strategy])
holdout_score += rmsle_holdout

print(' holdout score %.6f' % (rmsle_holdout))

/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/lightgbm/basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))
/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/lightgbm/basic.py:648: LGBMDeprecationWarning: The `max_bin` parameter is deprecated and will be removed in 2.0.12 version. Please use `params` to pass this parameter.
  'Please use `params` to pass this parameter.', LGBMDeprecationWarning)


 holdout score 0.515661
